# Gibbs sampling in Edward

## Boilerplate

In [1]:
import numpy as np
import tensorflow as tf
import edward as ed
from edward.models import Empirical, Beta, Bernoulli

In [2]:
# Adapted from https://stackoverflow.com/questions/38189119
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

## Model: Beta-Bernoulli

In [3]:
p = Beta(1.0, 1.0)
x = Bernoulli(probs=p, sample_shape=3)

Edward has already added to the tensorflow graph.

In [4]:
show_graph(tf.get_default_graph())

In [5]:
x_data = np.array([0, 1, 0])
qp = Empirical(tf.Variable(tf.zeros(10)))
inference = ed.Gibbs({p: qp}, data={x: x_data})

Edward has now added inference machinery to the tensorflow graph.

In [6]:
show_graph(tf.get_default_graph())

In [21]:
inference.run()

10/10 [100%] ██████████████████████████████ Elapsed: 0s | Acceptance Rate: 1.000
